In [1]:
from TT_Blip_architecture.classifier_layer import ClsfLayer
from TT_Blip_architecture.feature_extraction_layer import FeatureExtractionLayer
from TT_Blip_architecture.fusion_layer import FusionLayer
from TT_Blip_architecture.tt_blip import TT_Blip
from TT_Blip_architecture.data_processor import DataProcessor

In [2]:
import csv
from tqdm.auto import tqdm
import urllib.request


train_set = []
validation_set = []
with open("Fakeddit/multimodal_train.tsv") as fd:
    rd = csv.DictReader(fd, delimiter="\t", fieldnames=['author','clean_title','created_utc','domain','hasImage','id','image_url','linked_submission_id','num_comments','score','subreddit','title','upvote_ratio','2_way_label','3_way_label','6_way_label'])
    
    for line in tqdm(rd):
        train_set.append(line)
    train_set = train_set[1:]

with open("Fakeddit/multimodal_validate.tsv") as fd:
    rd = csv.DictReader(fd, delimiter="\t", fieldnames=['author','clean_title','created_utc','domain','hasImage','id','image_url','linked_submission_id','num_comments','score','subreddit','title','upvote_ratio','2_way_label','3_way_label','6_way_label'])
    
    for line in tqdm(rd):
        validation_set.append(line)
    validation_set = validation_set[1:]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [3]:
import urllib.request
import cv2 as cv
import numpy as np
from concurrent.futures import ThreadPoolExecutor

processor = DataProcessor()

def download_and_process_image(b):
    image_url = b['image_url']
    try:
        # Download the image data into memory
        with urllib.request.urlopen(image_url) as url_response:
            image_data = url_response.read()
            # Convert the byte data to a NumPy array
            image_array = np.asarray(bytearray(image_data), dtype=np.uint8)
            # Decode the image
            img = cv.imdecode(image_array, cv.IMREAD_COLOR)
            # Convert color space to RGB
            img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            # Get label and title
            y = int(b['2_way_label'])
            t = b['clean_title']
            return img, y, t
    except:
        # If any error occurs, return None
        return None

def collate_fn(batch):
    x = []
    y = []
    t = []
    # Use ThreadPoolExecutor to download and process images in parallel
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(download_and_process_image, b) for b in batch]
        for future in futures:
            result = future.result()
            if result is not None:
                img, label, title = result
                x.append(img)
                y.append(label)
                t.append(title)

    if(len(x) > 0):
        collate_fn.last_batch = (x, t, y)
        return processor(x, t, y)
    else:
        (x, y, t) = collate_fn.last_batch
        return processor(x, t, y)
    
collate_fn.last_batch = (0, 0, 0)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
from torch.utils.data import DataLoader

train_dl = DataLoader(train_set[5000:], batch_size=4, shuffle=True, collate_fn=collate_fn)
val_dl = DataLoader(validation_set[1000:], batch_size=4, shuffle=True, collate_fn=collate_fn)

next(iter(train_dl))

(tensor([[[[ 0.3961,  0.3804,  0.2784,  ..., -0.8824, -0.7882, -0.2392],
           [ 0.3882,  0.4118,  0.4353,  ..., -0.8275, -0.7333, -0.0510],
           [ 0.4275,  0.4824,  0.6000,  ..., -0.7569, -0.5686,  0.0980],
           ...,
           [ 0.0353,  0.1608,  0.3255,  ...,  0.8510,  0.7647,  0.8980],
           [-0.0902,  0.1059,  0.2471,  ...,  0.8745,  0.8039,  0.9137],
           [-0.0039,  0.2078,  0.2784,  ...,  0.8353,  0.8431,  0.9216]],
 
          [[-0.2863, -0.2471, -0.3020,  ..., -0.9765, -0.9373, -0.5059],
           [-0.2784, -0.1843, -0.1216,  ..., -0.9765, -0.9216, -0.3647],
           [-0.2235, -0.1294,  0.0353,  ..., -0.9451, -0.8196, -0.2627],
           ...,
           [-0.5922, -0.4824, -0.3412,  ..., -0.0824, -0.1686, -0.0353],
           [-0.7176, -0.5373, -0.4118,  ..., -0.0824, -0.1608, -0.0510],
           [-0.6471, -0.4353, -0.3804,  ..., -0.1373, -0.1529, -0.0667]],
 
          [[-0.6078, -0.5529, -0.6235,  ..., -0.9529, -0.9451, -0.5765],
           [-

In [5]:
feature_extraction = FeatureExtractionLayer()
fusion_layer = FusionLayer()
clsf_layer = ClsfLayer()

tt_blip = TT_Blip(feature_extraction, fusion_layer, clsf_layer)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from lightning import Trainer
from lightning.pytorch.loggers import WandbLogger

logger = WandbLogger(project="Thesis", name="TT-Blip Fakeddit")
trainer = Trainer(logger=logger, log_every_n_steps=1, max_epochs=50, accumulate_grad_batches=8)

trainer.fit(tt_blip, train_dl, val_dl)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: Currently logged in as: osusume. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                     | Type                   | Params | Mode 
----------------------------------------------------------------------------
0 | feature_extraction_layer | FeatureExtractionLayer | 443 M  | train
1 | fusion_layer             | FusionLayer            | 14.2 M | train
2 | clsf_layer               | ClsfLayer              | 5.9 M  | train
3 | loss_fn                  | BCEWithLogitsLoss      | 0      | train
4 | accuracy                 | BinaryAccuracy         | 0      | train
----------------------------------------------------------------------------
63.8 M    Trainable params
400 M     Non-trainable params
463 M     Total params
1,855.217 Total estimated model params size (MB)
154       Modules in train mode
832       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/daniele/Desktop/tt_blip_implementation/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:476: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/daniele/Desktop/tt_blip_implementation/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/daniele/Desktop/tt_blip_implementation/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]